In [1]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_2d89787c992c411eba59c89b454b0b6b"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_3debe6d3c07740a0bf5ac878429c9e67"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_29ff19f2389248fe99b1f33aa4d923f0"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_40d41d51625d41bb9c260e722bad8d29"},{"name":"output2","type":0,"uri":"tmp_829ae4e1d5ce4989a8bc7e350e649686"},{"name":"output3","type":0,"uri":"tmp_dd2bf7ee308e47e8a9c72c354e981910"},{"name":"output4","type":0,"uri":"tmp_0627f327349a4aea8b8d2cb5f85582d7"}]'

# 自定义参数


# 导入库

In [2]:
import wfio
import random
import math
import pandas as pd
import random
import numpy as np
import sys   
import re
from ustciscrBDL_B import get_score
import sklearn.tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib
import os

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# 导入数据

In [3]:
#读取训练集数据
def get_train_data():
    train_data1 = wfio.read_dataframe(_INPUT1)
    train_data2 = wfio.read_dataframe(_INPUT2)
    return train_data1.append(train_data2,ignore_index=True,sort=False)

In [4]:
#读取测试集数据
def get_test_data():
    test_data = wfio.read_dataframe(_INPUT3)
    return test_data

# 特征提取

In [5]:
season=['2017q1','2017q2','2017q3','2017q4',
        '2018q1','2018q2','2018q3','2018q4',
        '2019q1','2019q2','2019q3','2019q4']

In [6]:
#获取浮点数整数部分的长度
def num_length(num):
    return len(str(int(num)))

#字符串转浮点数（异常矫正）
def str2float(str):
    try:
        return float(str)
    except:
        return 0

# 生僻字风险度
def is_rare_name(string):
    risk=0
    pattern = re.compile(u"[~!@#$%^&* ]")
    match = pattern.search(string)
    if match:
        risk+=1

    try:
        string.encode("gb2312")
    except UnicodeEncodeError:
        risk+=1
    return risk

#登记开业时间转数字
def datestr2num(str):
    try:
        str=str.split('-')
        str[2]=str[2].split(' ')[0]
        num=(2019-int(str[0]))*365+(12-int(str[1]))*30+(30-int(str[2]))
        return num
    except:
        return 0

In [7]:
def feature_extract(dataframe,label=False):
    feature_dict={}
    
    keys=['fphdsl','jxje','jxsl','kpje','kps','kpse','rkse','kpsl'] # 开票数据、纳税数据
    
    for s in season:
        feature_dict[s+"_full_amount"]=[]   #开票顶额
        feature_dict[s+"_full_percent"]=[]   #开票顶额度
        #纳税开票情况
        for k in keys:
            feature_dict[s+k]=[]
        feature_dict[s+'jxse']=[]
        
    for s in range(len(season)-1):
        feature_dict[season[s]+"_increase"]=[]  #开票金额增量
        
    feature_dict["bsrxm_repeat"]=[] #重复注册bsrxm
    bsrxm_repeat=dataframe['bsrxm'].duplicated()
    
    feature_dict["fddbrxm_repeat"]=[] #重复注册fddbrxm
    fddbrxm_repeat=dataframe['fddbrxm'].duplicated()
    
    feature_dict["bsrxmmp_repeat"]=[] #重复注册bsrxmmp
    bsrxmmp_repeat=dataframe['bsrxmmp'].duplicated()
    
    feature_dict["fdbrxmp_repeat"]=[] #重复注册fdbrxmp
    fdbrxmp_repeat=dataframe['fdbrxmp'].duplicated()
    
    feature_dict["zcdz_repeat"]=[] #重复注册zcdz
    zcdz_repeat=dataframe['zcdz'].duplicated()
    
    feature_dict['djrq']=[] #登记日期
    
    feature_dict['gwhy']=[] #高危行业风险度
    hy_list=['hy','hydl','hyml','hyzl','jyfw']
    xy_list=['xy','xydl','xyml','xyzl','jyfw']
    gwhy_keys=["商","贸",'建材','批发','零售','咨询','鉴证','人力资源','服务','纺织','服饰','加工','中药','种植']
    
    feature_dict['name']=[] #命名规范
    
    feature_dict['location']=[] #公司地址风险度
    addr_pos=['小区','花园']
    addr_neg=["商",'办公','工']
    
    feature_dict['cyrs']=[] #从业人数
    
    feature_dict['zczb']=[] #注册资本
    
    if label:# 提取训练集需要'yc'
        feature_dict['yc']=[]
   
        
    for i in range(dataframe.shape[0]):
        
        for s in season:
            
            #开票顶额
            feature_dict[s+"_full_amount"].append(num_length(str2float(dataframe[s+"kpje"].iloc[i])))
            
            #开票顶额度
            feature_dict[s+"_full_percent"].append(str2float(dataframe[s+"kpje"].iloc[i])/10**num_length(str2float(dataframe[s+"kpje"].iloc[i])))
            
            
#             keys=['fphdsl','kps','jxje','kpje','kpse','rkse'] # 开票数据、纳税数据
            #纳税情况
            for k in keys:
                feature_dict[s+k].append(str2float(dataframe[s+k].iloc[i]))
                
            feature_dict[s+'jxse'].append(str2float(dataframe[s+'jxje'].iloc[i])*str2float(dataframe[s+'jxsl'].iloc[i]))
            
            
    
        for s in range(len(season)-1):
            #开票金额增量
            feature_dict[season[s]+"_increase"].append(str2float(dataframe[season[s+1]+'kpje'].iloc[i])-str2float(dataframe[season[s]+'kpje']))
        
        #重复注册bsrxm
        feature_dict["bsrxm_repeat"].append(float(bsrxm_repeat.iloc[i]))
        
        #重复注册fddbrxm
        feature_dict["fddbrxm_repeat"].append(float(fddbrxm_repeat.iloc[i])) 
        
        #重复注册bsrxmmp
        feature_dict["bsrxmmp_repeat"].append(float(bsrxmmp_repeat.iloc[i]))

        #重复注册fdbrxmp
        feature_dict["fdbrxmp_repeat"].append(float(fdbrxmp_repeat.iloc[i]))
        
        #重复注册zcdz
        feature_dict["zcdz_repeat"].append(float(zcdz_repeat.iloc[i]))
        
        #登记日期
        feature_dict["djrq"].append(datestr2num(dataframe['djrq']))
        
        #高危行业风险度
        risk=0
        try:
            for hy in hy_list:
                for w in gwhy_keys:
                    if w in dataframe[hy].iloc[i]:
                        risk+=1
        except:
            for hy in xy_list:
                for w in gwhy_keys:
                    if w in dataframe[hy].iloc[i]:
                        risk+=1
        feature_dict['gwhy'].append(risk)
        
        #公司命名风险度
        feature_dict['name'].append(is_rare_name(dataframe['nsrmc'].iloc[i]))
    
        # 地址风险度
        risk=0
        for l in ['scjydz','zcdz']:
            for p in addr_pos:
                if p not in dataframe[l].iloc[i]:
                    risk+=1
            for n in addr_neg:
                if n in dataframe[l].iloc[i]:
                    risk+=1
        feature_dict['location'].append(risk)
        
        # 从业人数
        feature_dict['cyrs'].append(str2float(dataframe['cyrs'].iloc[i]))
        
        #注册资本
        if 'zczby' in dataframe.keys():
            try:
                if math.isnan(float(dataframe['zczby'].iloc[i])):
                    feature_dict['zczb'].append(0)
                else:
                    feature_dict['zczb'].append(float(dataframe['zczby'].iloc[i]))
            except:
                feature_dict['zczb'].append(0)
        else:
            try:
                if math.isnan(float(dataframe['zczb'].iloc[i])):
                    feature_dict['zczb'].append(0)
                else:
                    feature_dict['zczb'].append(float(dataframe['zczb'].iloc[i]))
            except:
                feature_dict['zczb'].append(0)
        
        
        
        #标签
        if label:
            feature_dict['yc'].append(int(dataframe['yc'].iloc[i]))
            
    return feature_dict
        

# 数据转换

In [8]:
def data_tf(x,label=False):
    #x=data_normalize(x)
    temp=list(x.values())
    x=np.transpose(np.array(temp,np.float32))
    if label is False:
        return x
    else: # 训练集模式返回标签
        np.random.shuffle(x)
        x1=x[:18000]
        x2=x[18000:]
        return np.delete(x1, -1, axis=1),np.delete(x2, -1, axis=1),x1[:,-1],x2[:,-1]

# 分类算法

In [9]:
if not os.path.exists('tree.pkl'):
    train_data_=get_train_data()
    feature_dict=feature_extract(train_data_,True)
    train_feature,test_feature,train_label,test_label=data_tf(feature_dict,True)

In [10]:
if not os.path.exists('tree.pkl'):
    tree = DecisionTreeClassifier(criterion="entropy",
                                 splitter="best",
                                 max_depth=24,
                                 min_samples_split=2,
                                 min_samples_leaf=1,
                                 min_weight_fraction_leaf=0.,
                                 max_features=None,
                                 random_state=None,
                                 #max_leaf_nodes=26,
                                 min_impurity_decrease=0.,
                                 min_impurity_split=None,
                                 class_weight=None,
                                 presort=False
                                 )
    tree.fit(train_feature,train_label)
    print('Train score:{:.3f}'.format(tree.score(train_feature,train_label)))
    print('Test score:{:.3f}'.format(tree.score(test_feature,test_label)))
    joblib.dump(tree,'tree.pkl')
else:
    tree=joblib.load('tree.pkl')

Train score:0.997
Test score:0.996


In [11]:
test_data=get_test_data()
feature=feature_extract(test_data)
feature=data_tf(feature,False)

In [12]:
def get_verify_data_dt(feature,train_data):
    
    prob = {
        "zjnsrsbh": train_data['zjnsrsbh']
    }
    
    prob_list = []
    train_data = pd.DataFrame(prob)
    rows = train_data.shape[0]
    train_data['Probability'] = tree.predict_proba(feature)[:,1]
    return train_data

In [13]:
get_score.post_user_id('5')
user_verify_data_dt = get_verify_data_dt(feature,test_data)
get_score.post_verify_data(user_verify_data_dt)

User-id:5
程序开始运行时间为：2019-12-22 13:33:57.792356
程序结束运行时间为：2019-12-22 13:33:57.806553
程序运行时间（去除打分耗时）为：0.014197
程序的准确率为：93.43099483
最终得分为：94.08789535
